Adapted notebook from [Projection Path Explorer](https://github.com/JKU-ICG/projection-path-explorer/blob/7202a7d148f87269a3a9403ee1890679924546f7/notebooks/rubik_notebook.ipynb).

In [9]:
from matplotlib.pyplot import *
from sklearn import manifold
from numpy import *
from cube import *
import time

delete_duplicates = True

In [11]:
def val_from_colour(str):
    if str == 'W':
        return 0
    elif str == 'G':
        return 1
    elif str == 'R':
        return 2
    elif str == 'O':
        return 3
    elif str == 'Y':
        return 4
    elif str == 'B':
        return 5


def to_target_data(cube_data):
    # fill array with zeros
    # colors of a cubie described using one-hot: 1,0,0,0,0,0 might mean white=yes, red=no, blue=,...
    # there are 9 cubies per side
    # there a six sides per cube-setting
    # the number of cube-settings is len_array(cube_data[i])
    # there are several unique cubes being solved in several cube-settings

    output_array = []
    i = 0

    for cube in cube_data:
        for setting in cube:
            new_array = np.zeros(9 * 6 * 6)
            for side in setting:
                for row in side:
                    for val in row:
                        new_array[i + val_from_colour(val)] = 1
                        i += 6
            output_array.append(deepcopy(new_array))
            i = 0

    return output_array

In [12]:
# number of cubes to be solved using a simple algorithm with several checkpoints
# https://ruwix.com/the-rubiks-cube/how-to-solve-the-rubiks-cube-beginners-method/
loop_amt = 10
# if reuse_cube is True both algos are applied to the same random origin
reuse_cube = False
scramble_amt = 26
# at each position the solution process of a unique cube will be stored - all settings from random to solved
data = []
# same thing for the indices of the cubes that represent check points
check_points = []
algotype = []
differences = []

for i in range(loop_amt):
    print("scrambling "+str(i)+"...")
    scramble(scramble_amt)
    mvs = get_scramble()
    print("solving "+str(i)+"...")
    d, cp, diff = solve_simple()
    data.append(d)
    check_points.append(cp)
    algotype.append("0")
    differences.append(diff)
    print("amt cps", len(cp))

    print("scrambling " + str(i) + "...")
    if reuse_cube:
        scramble(mvs)
    else:
        scramble(scramble_amt)
    print("solving " + str(i) + "...")
    d, cp, diff = solve()
    data.append(d)
    check_points.append(cp)
    algotype.append("1")
    differences.append(diff)
    print("amt cps", len(cp))

for j in range(2 * loop_amt):
    print("checkpoints", check_points[j])
    print("data len", len(data[j]))
    data[j]
    sol_len = len(data)

cube_data = data

# solution of cube 1 lies in cube_data between the indices sol_len[0] and sol_len[1]
# solution of cube 2 lies in cube_data between the indices sol_len[1] and sol_len[2]
# ...
sol_len = [0]
for i in range(len(data)):
    print("len of cube:", len(data[i]))
    sol_len.append(sol_len[i] + len(data[i]))


# turning [[[['R', 'O', 'O'], ...] data structure into one-hot encoded data structure
used_data = to_target_data(cube_data)


print(len(used_data))

if delete_duplicates:
    used_data, inds, cs = np.unique(used_data, axis=0, return_inverse=True, return_counts=True)
    
print(len(used_data))

scrambling 0...
solving 0...
amt cps 4
scrambling 0...
solving 0...
amt cps 7
scrambling 1...
solving 1...
amt cps 4
scrambling 1...
solving 1...
amt cps 6
scrambling 2...
solving 2...
amt cps 6
scrambling 2...
solving 2...
amt cps 6
scrambling 3...
solving 3...
amt cps 4
scrambling 3...
solving 3...
amt cps 4
scrambling 4...
solving 4...
amt cps 4
scrambling 4...
solving 4...
amt cps 4
scrambling 5...
solving 5...
amt cps 5
scrambling 5...
solving 5...
amt cps 5
scrambling 6...
solving 6...
amt cps 4
scrambling 6...
solving 6...
amt cps 6
scrambling 7...
solving 7...
amt cps 5
scrambling 7...
solving 7...
amt cps 6
scrambling 8...
solving 8...
amt cps 5
scrambling 8...
solving 8...
amt cps 7
scrambling 9...
solving 9...
amt cps 4
scrambling 9...
solving 9...
amt cps 6
checkpoints [0, 16, 61, 120]
data len 121
checkpoints [0, 26, 58, 72, 80, 88, 100]
data len 101
checkpoints [0, 17, 54, 99]
data len 100
checkpoints [0, 21, 47, 54, 62, 88]
data len 89
checkpoints [0, 19, 53, 60, 69, 111

In [14]:
file = open("./cube.csv", "w")
header = "line,cp,algo"
sides = ["up", "front", "right", "left", "down", "back"]
for side in sides:
    for i in range(0,3):
        for j in range(0,3):
            header += ","+side+str(i)+str(j)
            
            
names = ['beginner','fridrich']
header += "\n"
# resulting header:
# x,y,cubeNum,up00,up01,up02,up10,up11,up12,up20,up21,up22,front00,front01,...
# x and y being coordinates of t-sne plot and up00 holding a letter for the color of the "up"-side top left cubie...
file.write(header)

cube_iterator = 0
i = 0
cp_i = 0
setting_i = 0
print(check_points)
for cube in cube_data:
    for si, setting in enumerate(cube):
        if check_points[cube_iterator][cp_i] == setting_i:
            cp_str = "checkpoint"
            cp_i += 1
        else:
            cp_str = "intermediate"
        
        if si == 0:
            cp_str = 'start'
        if si == len(cube) - 1:
            cp_str = 'end'
        
        file.write("%s, %s, %s" %(cube_iterator, cp_str, names[int(algotype[cube_iterator])]))
        for side in setting:
                for row in side:
                    for cubie in row:
                        file.write(",%s" %cubie)
        file.write("\n")
        i += 1
        setting_i += 1
    cube_iterator += 1
    setting_i = 0
    cp_i = 0
    
file.close()

[[0, 16, 61, 120], [0, 26, 58, 72, 80, 88, 100], [0, 17, 54, 99], [0, 21, 47, 54, 62, 88], [0, 19, 53, 60, 69, 111], [0, 25, 71, 79, 87, 99], [0, 20, 67, 135], [0, 25, 63, 86], [0, 17, 55, 98], [0, 15, 53, 79], [0, 25, 57, 63, 139], [0, 11, 58, 66, 109], [0, 24, 57, 100], [0, 30, 64, 71, 79, 106], [0, 25, 45, 52, 128], [0, 11, 43, 51, 59, 81], [0, 31, 62, 70, 105], [0, 34, 71, 82, 89, 97, 124], [0, 21, 54, 129], [0, 31, 64, 77, 85, 105]]


In [ ]:
test